
# **Exit Engine (from Array)** — apply grid-search exit rules to an in-memory `signals` list

Paste/construct your `signals` list (Python list of dicts) and run the exit evaluation — no CSV needed.
- **Entry**: next trading day **Open** after the signal's `bar_date`.
- **Exits**: **Stop Loss = 5%**, **Max Hold = 10 days** (calendar by default; toggleable).
- **Costs**: Groww-style + slippage (configurable).
- **Outputs**: in-memory DataFrames, optional CSV save, optional **Telegram alerts** for *today’s* exits.

### Expected `signals` schema (min fields)
```python
signals = [
  {"ticker": "RELIANCE.NS", "side": "long",  "bar_date": "2025-09-23"},
  {"ticker": "TCS.NS",      "side": "short", "bar_date": "2025-09-22"},
]
```
Optional per-signal overrides: `capital`, `stop_loss_pct`, `max_hold_days`.


In [29]:

# ===============================
# Configuration
# ===============================
import os, json, warnings, math, sys, requests
from typing import Dict, Any, List, Optional, Tuple
import numpy as np
import pandas as pd
import yfinance as yf

warnings.filterwarnings("ignore")

# Paste or programmatically construct your signals array here
signals: List[Dict[str, Any]] = [
    # Example:
    {"ticker": "ABFRL.NS", "side": "long",  "bar_date": "2025-09-04"},
    # {"ticker": "TCS.NS",      "side": "short", "bar_date": "2025-09-22", "capital": 60000},
]

# Evaluation date for deciding "exit today?"
# Set to IST 'today' for end-of-day checks; None = auto latest common bar across tickers
# EVAL_DATE: Optional[str] = pd.Timestamp.now(tz="Asia/Kolkata").strftime("%Y-%m-%d")
EVAL_DATE: Optional[str] = None

# === Exit Rule defaults (same as grid search baseline) ===
DEFAULT_STOP_LOSS_PCT: float = 0.05      # 5%
DEFAULT_MAX_HOLD_DAYS: int = 10          # 10 calendar days by default
DEFAULT_CAPITAL_PER_TRADE: float = 50000 # qty = floor(capital / entry_price)

# Count holding by 'calendar' days or by trading 'bars'
HOLD_DAYS_MODE: str = "calendar"         # 'calendar' (matches previous notebooks) or 'bars'

# === Costs & slippage (parameterized for Groww delivery; adjust if needed) ===
USE_COSTS: bool = True
SLIPPAGE_PCT_PER_LEG: float = 0.0005   # 0.05% per leg (buy & sell)
BROKERAGE_PER_LEG_RS: float = 0.0      # Groww delivery often 0
STT_SELL_PCT: float = 0.001            # 0.1% sell side
EXCHANGE_TXN_PCT: float = 0.0000345    # ~0.00345% turnover both legs
SEBI_PCT: float = 0.000001             # ~₹10 per crore
GST_PCT_ON_BROKERAGE: float = 0.18     # 18% GST on brokerage
STAMP_DUTY_BUY_PCT: float = 0.00015    # buy side stamp duty

# === Saving & Telegram ===
SAVE_OUTPUTS: bool = True
OUT_ROOT = "outputs/exits_from_array"
os.makedirs(OUT_ROOT, exist_ok=True)

SEND_TELEGRAM: bool = False
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "")
TELEGRAM_CHAT_ID   = os.getenv("TELEGRAM_CHAT_ID", "")

pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 200)


In [30]:

# ===============================
# Utilities
# ===============================
def validate_signals(rows: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    out = []
    for i, r in enumerate(rows):
        if not isinstance(r, dict):
            raise ValueError(f"signals[{i}] must be dict, got {type(r)}")
        for k in ("ticker","side","bar_date"):
            if k not in r:
                raise ValueError(f"signals[{i}] missing required key '{k}'")
        rr = r.copy()
        rr["ticker"] = str(rr["ticker"]).strip()
        rr["side"] = str(rr["side"]).lower().strip()
        rr["bar_date"] = str(rr["bar_date"]).strip()
        if rr["side"] not in ("long","short"):
            raise ValueError(f"signals[{i}] side must be 'long' or 'short'")
        out.append(rr)
    return out

def add_slippage(px: float, side_leg: str) -> float:
    if SLIPPAGE_PCT_PER_LEG <= 0: return px
    return px * (1.0 + SLIPPAGE_PCT_PER_LEG) if side_leg=='buy' else px * (1.0 - SLIPPAGE_PCT_PER_LEG)

def calc_costs(buy_val: float, sell_val: float) -> float:
    if not USE_COSTS:
        return 0.0
    brokerage_total = BROKERAGE_PER_LEG_RS * 2.0
    exch = EXCHANGE_TXN_PCT * (buy_val + sell_val)
    sebi = SEBI_PCT * (buy_val + sell_val)
    gst  = GST_PCT_ON_BROKERAGE * (BROKERAGE_PER_LEG_RS * 2.0)
    stt  = STT_SELL_PCT * sell_val
    stamp = STAMP_DUTY_BUY_PCT * buy_val
    return brokerage_total + exch + sebi + gst + stt + stamp

def download_ohlc(ticker: str, lookback_days: int = 800) -> pd.DataFrame:
    start = (pd.Timestamp.now(tz="Asia/Kolkata") - pd.Timedelta(days=lookback_days)).strftime("%Y-%m-%d")
    df = yf.download(ticker, start=start, end=None, interval="1d", progress=False, auto_adjust=True, multi_level_index=False)
    if df is None or df.empty: return pd.DataFrame()
    return df.dropna().copy()

def business_next_day(ts: pd.Timestamp, index: pd.DatetimeIndex) -> Optional[pd.Timestamp]:
    pos = index.searchsorted(ts + pd.Timedelta(days=0), side='right')
    if pos < len(index): return index[pos]
    return None

def choose_eval_date_from_data(frames: Dict[str, pd.DataFrame], eval_date: Optional[str]) -> Optional[pd.Timestamp]:
    if eval_date:
        return pd.to_datetime(eval_date)
    last = [df.index[-1].normalize() for df in frames.values() if not df.empty]
    if not last: return None
    return max(last)

def send_telegram_message(text: str, token: str, chat_id: str, parse_mode: str = "HTML"):
    if not token or not chat_id:
        print("Telegram token/chat_id not provided; skipping send.")
        return False
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {"chat_id": chat_id, "text": text, "parse_mode": parse_mode, "disable_web_page_preview": True}
    try:
        r = requests.post(url, data=payload, timeout=15)
        if r.status_code != 200:
            print("Telegram send error:", r.text)
            return False
        return True
    except Exception as e:
        print("Telegram send exception:", e)
        return False


In [31]:

# ===============================
# Core simulation
# ===============================
def simulate_exit_for_signal(sig: Dict[str, Any], eval_date: pd.Timestamp, df: pd.DataFrame) -> Dict[str, Any]:
    side = sig['side']
    bar_ts = pd.to_datetime(sig['bar_date'])
    capital = float(sig.get('capital', DEFAULT_CAPITAL_PER_TRADE))
    stop_loss_pct = float(sig.get('stop_loss_pct', DEFAULT_STOP_LOSS_PCT))
    max_hold_days = int(sig.get('max_hold_days', DEFAULT_MAX_HOLD_DAYS))

    # Entry on next trading day open
    entry_ts = business_next_day(bar_ts, df.index)
    if entry_ts is None:
        return dict(status="NO_ENTRY", reason="No future bar after signal", **sig)

    # Skip if eval_date before entry
    if eval_date.normalize() < entry_ts.normalize():
        return dict(status="WAIT", reason="Eval date before entry", entry_date=str(entry_ts.date()), **sig)

    entry_open = float(df.loc[entry_ts, 'Open'])
    entry_px = add_slippage(entry_open, 'buy' if side=='long' else 'sell')
    qty = int(capital // entry_px)
    if qty <= 0:
        return dict(status="SKIP", reason="Capital too small for 1 share", entry_date=str(entry_ts.date()), **sig)

    # Stop price per side
    stop_px = entry_px * (1.0 - stop_loss_pct) if side=='long' else entry_px * (1.0 + stop_loss_pct)

    # Iterate from entry bar up to eval_date inclusive
    idx_start = df.index.get_loc(entry_ts)
    idx_end   = df.index.searchsorted(eval_date.normalize(), side='right') - 1
    if idx_end < idx_start:
        idx_end = idx_start

    bars_held = 0
    hit_earlier = None
    exit_today  = None

    for j in range(idx_start, idx_end + 1):
        d = df.index[j]
        H = float(df['High'].iloc[j]); L = float(df['Low'].iloc[j]); C = float(df['Close'].iloc[j])

        # SL
        if side == 'long':
            if L <= stop_px:
                hit_earlier = (d, stop_px, "SL")
        else:
            if H >= stop_px:
                hit_earlier = (d, stop_px, "SL")

        # Holding measure
        if HOLD_DAYS_MODE == "calendar":
            held = int((d.normalize() - entry_ts.normalize()).days)
        else:  # bars
            if j == idx_start:
                bars_held = 0
            else:
                bars_held += 1
            held = bars_held

        # Time exit
        if held >= max_hold_days:
            if hit_earlier is None or d < hit_earlier[0]:
                hit_earlier = (d, C, "MAX_HOLD")

    # Already exited before today
    if hit_earlier is not None and hit_earlier[0].normalize() < eval_date.normalize():
        d_hit, px_hit, reason = hit_earlier
        buy_val  = entry_px * qty if side=='long' else px_hit * qty
        sell_val = px_hit   * qty if side=='long' else entry_px * qty
        fees = calc_costs(buy_val, sell_val) if USE_COSTS else 0.0
        gross = (px_hit - entry_px) * qty if side=='long' else (entry_px - px_hit) * qty
        net   = gross - fees
        return dict(
            status="EXITED_EARLIER",
            entry_date=str(entry_ts.date()), exit_date=str(d_hit.date()),
            entry_px=entry_px, exit_px=px_hit, qty=qty, stop_px=stop_px,
            reason=reason, gross_pnl=gross, fees=fees, net_pnl=net,
            days_held=held,
            **sig
        )

    # No trigger yet → OPEN, unless the trigger is exactly today
    if hit_earlier is None or hit_earlier[0].normalize() > eval_date.normalize():
        # still open
        if HOLD_DAYS_MODE == "calendar":
            curr_held = int((eval_date.normalize() - entry_ts.normalize()).days)
        else:
            curr_held = min(max(0, idx_end - idx_start), max_hold_days)
        return dict(
            status="OPEN",
            entry_date=str(entry_ts.date()), eval_date=str(eval_date.date()),
            entry_px=entry_px, qty=qty, stop_px=stop_px,
            days_held=curr_held,
            **sig
        )

    # Exit today
    d_hit, px_hit, reason = hit_earlier
    buy_val  = entry_px * qty if side=='long' else px_hit * qty
    sell_val = px_hit   * qty if side=='long' else entry_px * qty
    fees = calc_costs(buy_val, sell_val) if USE_COSTS else 0.0
    gross = (px_hit - entry_px) * qty if side=='long' else (entry_px - px_hit) * qty
    net   = gross - fees
    if HOLD_DAYS_MODE == "calendar":
        held_today = int((d_hit.normalize() - entry_ts.normalize()).days)
    else:
        held_today = min(max(0, idx_end - idx_start), max_hold_days)
    return dict(
        status="EXIT_TODAY",
        entry_date=str(entry_ts.date()), exit_date=str(d_hit.date()),
        entry_px=entry_px, exit_px=px_hit, qty=qty, stop_px=stop_px,
        reason=reason, gross_pnl=gross, fees=fees, net_pnl=net,
        days_held=held_today,
        **sig
    )


In [32]:

# ===============================
# Run
# ===============================
signals = validate_signals(signals)
if not signals:
    raise ValueError("Please provide at least one signal in the 'signals' list.")

tickers = sorted({s['ticker'] for s in signals})
frames = {t: download_ohlc(t) for t in tickers}
frames = {t: df for t, df in frames.items() if not df.empty}
if not frames:
    raise RuntimeError("No OHLC data fetched. Check tickers/connectivity.")

chosen_eval_ts = choose_eval_date_from_data(frames, EVAL_DATE)
if chosen_eval_ts is None:
    raise RuntimeError("Could not determine evaluation date.")
print("Evaluation date:", chosen_eval_ts.date())

rows = []
for sig in signals:
    df = frames.get(sig['ticker'])
    if df is None:
        rows.append(dict(status="NO_DATA", reason="OHLC not available", **sig))
        continue
    res = simulate_exit_for_signal(sig, chosen_eval_ts, df)
    rows.append(res)

status_df = pd.DataFrame(rows)
exit_today = status_df[status_df['status'] == 'EXIT_TODAY'].copy()

display(status_df.head())

if SAVE_OUTPUTS:
    out_dir = os.path.join(OUT_ROOT, chosen_eval_ts.strftime("%Y-%m-%d"))
    os.makedirs(out_dir, exist_ok=True)
    status_path = os.path.join(out_dir, "positions_status_from_array.csv")
    exit_path   = os.path.join(out_dir, "exit_today_from_array.csv")
    status_df.to_csv(status_path, index=False)
    exit_today.to_csv(exit_path, index=False)
    print("Saved:", status_path)
    print("Saved:", exit_path)

if SEND_TELEGRAM and not exit_today.empty:
    parts = [f"<b>Exit Alerts</b> — {chosen_eval_ts.strftime('%Y-%m-%d')}"]
    for _, r in exit_today.sort_values(['ticker']).iterrows():
        parts.append(
            f"• <b>{r['ticker']}</b> {r['side']} — exit @ {r.get('exit_px', float('nan')):.2f} "
            f"({r.get('reason','')}); P&L: {r.get('net_pnl', float('nan')):.2f} | Held {int(r.get('days_held',0))}d"
        )
    msg = "\n".join(parts)
    ok = True
    if len(msg) > 3800:
        lines, cur = msg.splitlines(), ""
        for ln in lines:
            if len(cur) + len(ln) + 1 > 3800:
                ok = ok and send_telegram_message(cur, TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID)
                cur = ln
            else:
                cur = (cur + "\n" + ln) if cur else ln
        if cur:
            ok = ok and send_telegram_message(cur, TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID)
    else:
        ok = send_telegram_message(msg, TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID)
    if not ok:
        print("One or more Telegram messages failed to send.")
else:
    print("No exit alerts to send or SEND_TELEGRAM=False.")


Evaluation date: 2025-09-24


,status,entry_date,exit_date,entry_px,exit_px,qty,stop_px,reason,gross_pnl,fees,net_pnl,days_held,ticker,side,bar_date
0,EXITED_EARLIER,2025-09-05,2025-09-15,83.371667,87.339996,599,79.203083,MAX_HOLD,2377.029374,63.4377,2313.591674,19,ABFRL.NS,long,2025-09-04


Saved: outputs/exits_from_array/2025-09-24/positions_status_from_array.csv
Saved: outputs/exits_from_array/2025-09-24/exit_today_from_array.csv
No exit alerts to send or SEND_TELEGRAM=False.
